In [1]:
import acquire_minecraft_urls as amu

In [2]:
# import personal modules
import prepare as prep
import acquire as ac


# import modules from libraries
#from prepare import basic_clean, lemmatize
from pprint import pprint

#import datascience libraries
import pandas as pd
import numpy as np

# import vizualization libraries
import matplotlib.pyplot as plt

# Sklearn modules including classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier  # Gradient Boosting Classifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB  # Naive Bayes Classifier


# Sklearn testing, evaluating, and managing model
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE, f_regression
from sklearn.multioutput import MultiOutputClassifier as MOC 
from sklearn.pipeline import Pipeline as pipeline 

# more classifiers
from xgboost import XGBClassifier  # XG Boost Classifier
from lightgbm import LGBMClassifier # Light Gradient Boost Classifier


import nltk #Natural Language Tool Kit
import re   #Regular Expressions

# NLP related modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

ModuleNotFoundError: No module named 'env'